In [59]:
import os
import numpy as np
import keras
from keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

In [6]:
train_dir = r'./dataset/RiceLeafs/train'
test_dir = r'./dataset/RiceLeafs/validation'

In [3]:
classes=['Hispa', 'Healthy', 'BrownSpot', 'LeafBlast']
indices = range(len(classes))
normal_mapping = dict(zip(classes, indices))
reverse_mapping = dict(zip(indices, classes))

In [52]:
training_data = []
class_index = 0

for file in os.listdir(train_dir):
    path = os.path.join(train_dir, file)
    for im in os.listdir(path):
        image = load_img(os.path.join(path, im), grayscale=False, color_mode='rgb', target_size=(80,80))
        image = img_to_array(image)
        image /= 255
        training_data += [[image,class_index]]
    class_index += 1

In [53]:
test_data = []
class_index = 0

for file in os.listdir(test_dir):
    path = os.path.join(test_dir, file)
    for im in os.listdir(path):
        image = load_img(os.path.join(path, im), grayscale=False, color_mode='rgb', target_size=(80,80))
        image = img_to_array(image)
        image /= 255
        test_data += [[image,class_index]]
    class_index += 1

In [54]:
training_images, training_labels = zip(*training_data)
testing_images, testing_labels = zip(*test_data)

training_labels = np.array(to_categorical(training_labels))
training_images = np.array(training_images)
testing_images = np.array(testing_images)

In [55]:
x_train,x_test,y_train,y_test = train_test_split(training_images,training_labels,test_size=0.2,random_state=44)
y_train = np.array(y_train)
y_test = np.array(y_test)
x_train = np.array(x_train)
x_test = np.array(x_test)
print(y_train.shape)
print(y_test.shape)
print(x_train.shape)
print(x_test.shape)

(1878, 4)
(806, 4)
(1878, 80, 80, 3)
(806, 80, 80, 3)


In [56]:
datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    fill_mode="nearest",
    # brightness_range=[0.8, 1.2],  # Slight color jitter
    # channel_shift_range=20  # Slight color jitter
)

In [57]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4, activation='softmax')
], name='Conv_Model_Enhanced')

optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "Conv_Model_Enhanced"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 78, 78, 32)        896       
                                                                 
 batch_normalization_15 (Ba  (None, 78, 78, 32)        128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 39, 39, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 37, 37, 64)        18496     
                                                                 
 batch_normalization_16 (Ba  (None, 37, 37, 64)        256       
 tchNormalization)                                               
                                               

In [73]:
def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = BatchNormalization()(x)
    x = Add()([shortcut, x])
    x = ReLU()(x)
    return x

input = Input(shape=(80, 80, 3))
x = Conv2D(64, (7, 7), padding="same", strides=2)(input)
x = BatchNormalization()(x)
x = ReLU()(x)

x = residual_block(x, 64)
x = residual_block(x, 64)

x = GlobalAveragePooling2D()(x)
x = Dense(4, activation="softmax")(x)  # Assuming 4 classes for classification

model = keras.Model(inputs=input, outputs=x)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 conv2d_23 (Conv2D)          (None, 40, 40, 64)           9472      ['input_10[0][0]']            
                                                                                                  
 batch_normalization_26 (Ba  (None, 40, 40, 64)           256       ['conv2d_23[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_5 (ReLU)              (None, 40, 40, 64)           0         ['batch_normalization_26

In [66]:
pretrained_model = keras.applications.DenseNet121(input_shape=(80,80,3),
                                                      include_top=False,
                                                      weights='imagenet')
inputs = pretrained_model.input
x = keras.layers.Flatten()(pretrained_model.output)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(4, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 zero_padding2d_8 (ZeroPadd  (None, 86, 86, 3)            0         ['input_6[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv1/conv (Conv2D)         (None, 40, 40, 64)           9408      ['zero_padding2d_8[0][0]']    
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 40, 40, 64)           256       ['conv1/conv[0][0]']    

In [71]:
base_model = keras.applications.DenseNet201(input_shape=(80, 80, 3), include_top=False, weights='imagenet')
model = keras.models.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(4, activation='softmax')  # Assuming 4 classes for classification
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 2, 2, 1920)        18321984  
                                                                 
 global_average_pooling2d_2  (None, 1920)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_29 (Dense)            (None, 4)                 7684      
                                                                 
Total params: 18329668 (69.92 MB)
Trainable params: 18100612 (69.05 MB)
Non-trainable params: 229056 (894.75 KB)
_________________________________________________________________


In [74]:
his=model.fit(datagen.flow(x_train,y_train,batch_size=32),validation_data=(x_test,y_test),epochs=15)

Epoch 1/15
59/59 [==============================] - 19s 280ms/step - loss: 1.0507 - accuracy: 0.5719 - val_loss: 1.1544 - val_accuracy: 0.4603
Epoch 2/15
59/59 [==============================] - 23s 391ms/step - loss: 0.9877 - accuracy: 0.6033 - val_loss: 1.7199 - val_accuracy: 0.2444
Epoch 3/15
59/59 [==============================] - 22s 366ms/step - loss: 0.9850 - accuracy: 0.5980 - val_loss: 1.6755 - val_accuracy: 0.1811
Epoch 4/15
59/59 [==============================] - 21s 354ms/step - loss: 0.9699 - accuracy: 0.6038 - val_loss: 1.0195 - val_accuracy: 0.6290
Epoch 5/15
59/59 [==============================] - 24s 410ms/step - loss: 0.9707 - accuracy: 0.6118 - val_loss: 0.9715 - val_accuracy: 0.6340
Epoch 6/15
59/59 [==============================] - 25s 419ms/step - loss: 0.9715 - accuracy: 0.6092 - val_loss: 2.3351 - val_accuracy: 0.2556
Epoch 7/15
59/59 [==============================] - 25s 416ms/step - loss: 0.9649 - accuracy: 0.6145 - val_loss: 1.1540 - val_accuracy: 0.5347

In [75]:
model.save("resnet60.h5")

/home/harshvardhanpandey/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
